In [ ]:
import pandas as pd
from os import walk
import matplotlib.pyplot as plt
from CritRole.Common import EDamageTypes
from CritRole.Common import validOrZero
import numpy as np

globalData = {}
accmData = []

gindex = 0

def process_file(root, x, character):
    global gindex
    fileindex = x.split('.')[0].zfill(3)
#    if not x == "64.html.csv":# and not x == "44.html.csv" and not x == "33.html.csv":
#       return
    #print("Processing file: " + str(x))
    eviFrame = pd.read_csv(root + x, index_col=0);
    retable = eviFrame.index.values
    #print(eviFrame["PhysicalInstVals"][0:20])
    #eviFrame = eviFrame.loc[:, EDamageTypes[0]:EDamageTypes[len(EDamageTypes) - 1]+"Instances"]
    eviFrame = eviFrame.loc[:, EDamageTypes[0]:EDamageTypes[len(EDamageTypes) - 1]]
    eviFrame = eviFrame.iloc[[(character)]]
    #eviFrame = eviFrame.rename(index={1: gindex})
    eviFrame.index.name = "Episode"
    last = eviFrame.index[-1]
    eviFrame = eviFrame.rename(index={last: str(fileindex)})
    #eviFrame["Episode"] = fileindex
    gindex += 1
    # + eviFrame[2:8]
    #eviFrame = eviFrame.drop(['Trinket'])
    #print(eviFrame)
    
    #print(character)
    if len(accmData) > character:
        #accmData[0].append(eviFrame, ignore_index=True);
        accmData[character] = pd.concat([accmData[character], eviFrame]);
        #print(gindex)
        #print(accmData[0])
    else:
        accmData.append(eviFrame);
        #print(accmData[0])
        

    #print(eviFrame)
    #print(accmData)
#    print(eviFrame.shape[0])
    for i in range(0, len(EDamageTypes)):
        #print(eviFrame.columns[i]);
        dAgg = eviFrame[eviFrame.columns[i]];

        if EDamageTypes[i] in globalData:
            globalData[EDamageTypes[i]][0] += dAgg
        else:
            globalData[EDamageTypes[i]] = []
            globalData[EDamageTypes[i]].append(dAgg)
#        DamageAgg = eviFrame[''
    return retable
        
# Enumerate the folder
damageTakens = []
rootDirectory = "./ParsedDamageTaken/"
for (dirpath, dirnames, filenames) in walk(rootDirectory):
    damageTakens.extend(filenames)

chararr = []
    
for chars in range(0,9):
    for singleFile in damageTakens:
        chararr = process_file(rootDirectory, singleFile, chars)
        
    accmData[chars] = accmData[chars].sort_index()
    print("======= CHAR " + str(chars) + " ========")
    print(accmData[chars])
        
print("Done")


In [ ]:
import matplotlib.colors as mplc
# Lightning  Poison  Fire  Falling  Radiant  Necrotic  Psychic  Ice  Acid  Thunder  Physical  
colorsList = [
    (1,1,0),
    (0.05,0.5,0.05),
    (1,0.25,0),
    (0.5,0.5,0.7),
    (1,0.8,0.05), # Radiant
    (0.5,0.25,0),
    (0.8,0.0,0.8), #Phychic
    (0.5,0.7,1.0),
    (0,1,0), #Acid
    (1,1,1),
    (0.9,0.9,1.0)
]
CustomCmap = mplc.ListedColormap(colorsList)

In [ ]:
def parseTotal(x):
    print(x)

In [ ]:
for i in range(0, len(accmData)):
    accmData[i]["Collect"] = accmData[i].apply(np.sum, axis=1)
    #loc[:,"To":"Effect"]

In [ ]:
#frames
framelist = []
ev = pd.DataFrame()
for i in range(0,len(accmData)):
    framelist.append(accmData[i])
    ev[chararr[i]] = accmData[i]["Collect"]
    
# use
#ev = pd.concat(accmData, axis=1)
ev

In [ ]:
ll = plt.figure(dpi=99)#;, facecolor='w', edgecolor='k')
#plt.figure(figsize=(20,10))
indexvalues = [int(x) for x in accmData[i].index.values]
#gplt = accmData[0].plot.area(stacked=True, figsize=(48,27), colormap="tab20c")
gplt = ev.plot.bar(ax = plt.gca(), stacked=True, figsize=(48,27), colormap=CustomCmap, xticks=indexvalues)
gplt.set_facecolor('#121212')
gplt.plot()
plt.savefig("out/campaignDamageTakenPerEpisode.taken.png")

In [ ]:
ll = plt.figure(dpi=99)#;, facecolor='w', edgecolor='k')
#plt.figure(figsize=(20,10))
indexvalues = [int(x) for x in accmData[i].index.values]
#gplt = accmData[0].plot.area(stacked=True, figsize=(48,27), colormap="tab20c")
gplt = ev.plot.bar(ax = plt.gca(), stacked=False, figsize=(48,27), colormap=CustomCmap, xticks=indexvalues)
gplt.set_facecolor('#121212')
gplt.plot()
plt.savefig("out/campaignDamageTakenPerEpisode.alt.barnostack.png")